In [1]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle
import random
import pdb
from torch.utils.data import DataLoader
import logging
import itertools
import unicodedata
import string
import re
import random
import argparse
from torch import optim
import time
import os
from torch.utils.data import BatchSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import Sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import math
import copy
import seaborn
import datetime
from logger import Logger

from torch.optim import RMSprop
import shutil
from statistics import mean

from gym_wrappers import MainGymWrapper
import gym

/home/cvh255/atari_game/logger.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/traitlets-4.3.2-py3.6.egg/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel/kernelapp.py"

In [2]:
device = torch.device('cpu')

In [3]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self, input_shape, action_space):
        super(ConvolutionalNeuralNetwork, self).__init__()
        self.input_shape = input_shape
        self.action_space = action_space
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=32, kernel_size=8, stride=(4,4))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,kernel_size=4,stride=(2,2))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        
        o_size = 3136
        self.linear1 = nn.Linear(in_features=o_size, out_features=512)
        self.linear2 = nn.Linear(in_features=512, out_features=self.action_space)
        
    def forward(self, x, batch_size=None):
        x = torch.from_numpy(x).float().to(device)
#         print(x.size(), torch.Size([1, 4, 84, 84]))
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))

        # out = (bs, nc, x, y)
        if batch_size==None:
            batch_size = out.size(0)
#         print(out.size())
        out_flat = out.view(batch_size, -1)
        
        out_flat = F.relu(self.linear1(out_flat))
        
        out_flat = self.linear2(out_flat)
        return out_flat

In [4]:
game_name = "Breakout-v0"
game_mode = 'ddqn_training'
render = False
total_step_limit = 5000000
total_run_limit = None
clip = True

FRAMES_IN_OBSERVATION = 4
FRAME_SIZE = 84
INPUT_SHAPE = (FRAMES_IN_OBSERVATION, FRAME_SIZE, FRAME_SIZE)

In [5]:
env_name = game_name + "Deterministic-v4" 
game_name, env_name

('Breakout-v0', 'Breakout-v0Deterministic-v4')

In [6]:
env = MainGymWrapper.wrap(gym.make(game_name))

In [7]:
GAMMA = 0.99
MEMORY_SIZE = 900000
BATCH_SIZE = 32
TRAINING_FREQUENCY = 4
TARGET_NETWORK_UPDATE_FREQUENCY = 40000
MODEL_PERSISTENCE_UPDATE_FREQUENCY = 10000
REPLAY_START_SIZE = 50000

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.1
EXPLORATION_TEST = 0.02
EXPLORATION_STEPS = 850000
EXPLORATION_DECAY = (EXPLORATION_MAX-EXPLORATION_MIN)/EXPLORATION_STEPS

In [8]:
class DDQNTrainer:
    def __init__(self, game_name, input_shape, action_space):
        
        self.input_shape = input_shape
        self.game_name = game_name
        self.action_space = action_space
        
        self.game_mode = "DDQN training"
        self.model_path = "/scratch/cvh255/atari/output/neural_nets/" + game_name + "/actual_ddqn/" + self._get_date() + "/model.h5"
#         self.model_path
        self.logger_path = "/scratch/cvh255/atari/output/logs/" + game_name + "/actual_ddqn/training/" + self._get_date() + "/"
        self.logger = Logger(self.game_name + " " + self.game_mode, self.logger_path)
        
        if os.path.exists(os.path.dirname(self.model_path)):
            shutil.rmtree(os.path.dirname(self.model_path), ignore_errors=True)
        os.makedirs(os.path.dirname(self.model_path))

        self.ddqn = ConvolutionalNeuralNetwork(self.input_shape, action_space).to(device)
        
        self.criteria = nn.MSELoss()
        self.optimizer = RMSprop(self.ddqn.parameters(), lr=0.00025, alpha = 0.95, eps=0.01)
        
        if os.path.isfile(self.model_path):
#             self.ddqn.load_state_dict(torch.load(self.model_path))
            self.ddqn.load_state_dictict(torch.load("/home/cvh255/atari_game/output/neural_nets/Breakout-v0try4/actual_ddqn/2019-05-07/model.h5"))
        
        self.ddqn_target = ConvolutionalNeuralNetwork(self.input_shape, action_space).to(device)
#         .model
        self._reset_target_network()
        self.epsilon = EXPLORATION_MAX
        self.memory = []
    
    def _save_model(self):
        torch.save(self.ddqn.state_dict(),self.model_path)
        
    def _get_date(self):
        return str(datetime.datetime.now().strftime('%Y-%m-%d'))
        
    def save_run(self, score, step, run):
        self.logger.add_score(score)
        self.logger.add_step(step)
        self.logger.add_run(run)

    def move(self, state):
        if np.random.rand() < self.epsilon or len(self.memory) < REPLAY_START_SIZE:
            return random.randrange(self.action_space)
        
        q_values = self.ddqn(np.expand_dims(np.asarray(state).astype(np.float64), axis=0), batch_size=1)
        return np.argmax(q_values[0].detach().cpu().numpy())

    def remember(self, current_state, action, reward, next_state, terminal):
        self.memory.append({"current_state": current_state,
                            "action": action,
                            "reward": reward,
                            "next_state": next_state,
                            "terminal": terminal})
        if len(self.memory) > MEMORY_SIZE:
            self.memory.pop(0)

    def step_update(self, total_step):
        if len(self.memory) < REPLAY_START_SIZE:
            return

        if total_step % TRAINING_FREQUENCY == 0:
            loss, accuracy, average_max_q = self._train()
            self.logger.add_loss(loss.item())
            self.logger.add_accuracy(accuracy)
            self.logger.add_q(average_max_q)

        self._update_epsilon()

        if total_step % MODEL_PERSISTENCE_UPDATE_FREQUENCY == 0:
            self._save_model()

        if total_step % TARGET_NETWORK_UPDATE_FREQUENCY == 0:
            self._reset_target_network()
            print('{{"metric": "epsilon", "value": {}}}'.format(self.epsilon))
            print('{{"metric": "total_step", "value": {}}}'.format(total_step))

    def _train(self):
        
        self.optimizer.zero_grad()
        self.ddqn_target.eval()
        batch = np.asarray(random.sample(self.memory, BATCH_SIZE))
        if len(batch) < BATCH_SIZE:
            return

        current_states = []
        actions = []
        rewards = []
        next_states = []
        terminals = []
        
        for entry in batch:
            current_state = np.expand_dims(np.asarray(entry["current_state"]).astype(np.float64), axis=0)
            current_states.append(current_state)
            
            next_state = np.expand_dims(np.asarray(entry["next_state"]).astype(np.float64), axis=0)
            next_states.append(next_state)
            
            rewards.append(entry['reward'])
            actions.append(entry['action'])
            terminals.append(entry['terminal'])
            
        non_final_mask = torch.tensor(terminals)==False
        non_final_next_states = np.asarray(next_states).squeeze()
        
        state_batch = np.asarray(current_states).squeeze()
        action_batch = np.asanyarray(actions).squeeze()
        reward_batch = np.asarray(rewards).squeeze() 
        
        state_action_values = self.ddqn(state_batch).gather(1, torch.from_numpy(action_batch).unsqueeze(dim=1).to(device))
        ddqn_next_state_out = self.ddqn(non_final_next_states)
        ddqn_next_actions = torch.argmax(ddqn_next_state_out, dim=1)
        
#         print(ddqn_next_actions.size(), 32)
        
        next_state_values = np.zeros(BATCH_SIZE)
        temp = self.ddqn_target(non_final_next_states).gather(1, ddqn_next_actions.unsqueeze(dim=1).to(device)).squeeze().detach().cpu().numpy()
#         print("tem var",temp.shape, 32)
        
        next_state_values[list(np.array(terminals)==False)] = temp[list(np.array(terminals)==False)]
        expected_state_action_values = (next_state_values * GAMMA) + reward_batch
        
#         loss = self.criteria(state_action_values, torch.from_numpy(expected_state_action_values).unsqueeze(1))
        loss = F.smooth_l1_loss(state_action_values, torch.from_numpy(expected_state_action_values).unsqueeze(1).to(device).float())
        loss.backward()
        
        for param in self.ddqn.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()
        
#         print(torch.from_numpy(action_batch).unsqueeze(dim=1).size(),torch.Size([32, 1]))
#         print(state_action_values.size(),[32, 4])
#         print(state_action_values.size(),torch.Size([32, 1]))
#         print(non_final_next_states.squeeze().size(), torch.Size([32, 4, 84, 84]))
        
        accuracy = 0
        return loss, accuracy, mean(expected_state_action_values)

    def _update_epsilon(self):
        self.epsilon -= EXPLORATION_DECAY
        self.epsilon = max(EXPLORATION_MIN, self.epsilon)

    def _reset_target_network(self):
        self.ddqn_target.load_state_dict(self.ddqn.state_dict())

In [9]:
def main_loop(game_model, env, render, total_step_limit, total_run_limit, clip):
    run = 0
    total_step = 0
    while True:
        if run%10==0:
            print('-'*100)
            print("RUN = ",run)
        if total_run_limit is not None and run >= total_run_limit:
            print("Reached total run limit of: " + str(total_run_limit))
            exit(0)

        run += 1
        current_state = env.reset()
        step = 0
        score = 0

        while True:
            if total_step >= total_step_limit:
                print("Reached total step limit of: " + str(total_step_limit))
                exit(0)
            total_step += 1
            
            step += 1

            if render:
                env.render()

            action = game_model.move(current_state)
            next_state, reward, terminal, info = env.step(action)
            if clip:
                np.sign(reward)
            score += reward
            game_model.remember(current_state, action, reward, next_state, terminal)
            current_state = next_state

            game_model.step_update(total_step)

            if terminal:
                game_model.save_run(score, step, run)
                break

In [10]:
game_model = DDQNTrainer(game_name+"try5", INPUT_SHAPE, env.action_space.n)
main_loop(game_model, env, render, total_step_limit, total_run_limit, clip)

----------------------------------------------------------------------------------------------------
RUN =  0


/home/cvh255/pyenv/gym/gym/logger.py:30: UserWarning: WARN: <class 'gym_wrappers.NoopResetEnv'> doesn't implement 'step' method, which is required for wrappers derived directly from Wrapper. Deprecated default implementation is used.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


score: (min: 0.0, avg: 0.9, max: 2.0
{"metric": "score", "value": 0.9}
step: (min: 164, avg: 214.5, max: 275
{"metric": "step", "value": 214.5}
{"metric": "run", "value": 10}
----------------------------------------------------------------------------------------------------
RUN =  10
score: (min: 0.0, avg: 1.5, max: 3.0
{"metric": "score", "value": 1.5}
step: (min: 164, avg: 252.5, max: 362
{"metric": "step", "value": 252.5}
{"metric": "run", "value": 20}
----------------------------------------------------------------------------------------------------
RUN =  20
score: (min: 0.0, avg: 1.9, max: 5.0
{"metric": "score", "value": 1.9}
step: (min: 167, avg: 270.7, max: 429
{"metric": "step", "value": 270.7}
{"metric": "run", "value": 30}
----------------------------------------------------------------------------------------------------
RUN =  30
score: (min: 0.0, avg: 0.7, max: 3.0
{"metric": "score", "value": 0.7}
step: (min: 166, avg: 203.8, max: 326
{"metric": "step", "value": 203.8

step: (min: 169, avg: 278.1, max: 374
{"metric": "step", "value": 278.1}
{"metric": "run", "value": 260}
----------------------------------------------------------------------------------------------------
RUN =  260
loss: (min: 0.0007094059837982059, avg: 0.4401199226795579, max: 2.599966049194336
{"metric": "loss", "value": 0.4401199226795579}
accuracy: (min: 0, avg: 0, max: 0
{"metric": "accuracy", "value": 0}
q: (min: -1.8207594629377126, avg: 0.09398872013221961, max: 1.2377051301673054
{"metric": "q", "value": 0.09398872013221961}
score: (min: 0.0, avg: 1.8, max: 5.0
{"metric": "score", "value": 1.8}
step: (min: 162, avg: 276.2, max: 443
{"metric": "step", "value": 276.2}
{"metric": "run", "value": 270}
----------------------------------------------------------------------------------------------------
RUN =  270
score: (min: 0.0, avg: 1.3, max: 3.0
{"metric": "score", "value": 1.3}
step: (min: 166, avg: 239.1, max: 325
{"metric": "step", "value": 239.1}
{"metric": "run", "value"

loss: (min: 1.998965126404073e-05, avg: 0.005109794764004618, max: 0.06056881323456764
{"metric": "loss", "value": 0.005109794764004618}
accuracy: (min: 0, avg: 0, max: 0
{"metric": "accuracy", "value": 0}
q: (min: 0.8711850347556174, avg: 0.9879620485286601, max: 1.0799004416354
{"metric": "q", "value": 0.9879620485286601}
score: (min: 0.0, avg: 1.2, max: 4.0
{"metric": "score", "value": 1.2}
step: (min: 171, avg: 235, max: 397
{"metric": "step", "value": 235}
{"metric": "run", "value": 430}
----------------------------------------------------------------------------------------------------
RUN =  430


KeyboardInterrupt: 

In [ ]:
game_model = DDQNTrainer(game_name+"try4", INPUT_SHAPE, env.action_space.n)
main_loop(game_model, env, render, total_step_limit, total_run_limit, clip)

----------------------------------------------------------------------------------------------------
RUN =  0


/home/cvh255/pyenv/gym/gym/logger.py:30: UserWarning: WARN: <class 'gym_wrappers.NoopResetEnv'> doesn't implement 'step' method, which is required for wrappers derived directly from Wrapper. Deprecated default implementation is used.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


score: (min: 0.0, avg: 2.0, max: 9.0
{"metric": "score", "value": 2.0}
step: (min: 174, avg: 266.4, max: 532
{"metric": "step", "value": 266.4}
{"metric": "run", "value": 10}
----------------------------------------------------------------------------------------------------
RUN =  10
score: (min: 0.0, avg: 1.3, max: 4.0
{"metric": "score", "value": 1.3}
step: (min: 172, avg: 241.5, max: 348
{"metric": "step", "value": 241.5}
{"metric": "run", "value": 20}
----------------------------------------------------------------------------------------------------
RUN =  20
score: (min: 0.0, avg: 1.9, max: 4.0
{"metric": "score", "value": 1.9}
step: (min: 170, avg: 275.6, max: 378
{"metric": "step", "value": 275.6}
{"metric": "run", "value": 30}
----------------------------------------------------------------------------------------------------
RUN =  30
score: (min: 0.0, avg: 1.4, max: 4.0
{"metric": "score", "value": 1.4}
step: (min: 163, avg: 255.7, max: 420
{"metric": "step", "value": 255.7

step: (min: 147, avg: 277.6, max: 634
{"metric": "step", "value": 277.6}
{"metric": "run", "value": 260}
----------------------------------------------------------------------------------------------------
RUN =  260
loss: (min: 0.0012620590860024095, avg: 0.6741222629882395, max: 3.626967668533325
{"metric": "loss", "value": 0.6741222629882395}
accuracy: (min: 0, avg: 0, max: 0
{"metric": "accuracy", "value": 0}
q: (min: -1.579737607575953, avg: 0.13042048124039546, max: 2.7319128481298685
{"metric": "q", "value": 0.13042048124039546}
score: (min: 0.0, avg: 1.8, max: 7.0
{"metric": "score", "value": 1.8}
step: (min: 162, avg: 262.6, max: 436
{"metric": "step", "value": 262.6}
{"metric": "run", "value": 270}
----------------------------------------------------------------------------------------------------
RUN =  270
score: (min: 0.0, avg: 1.5, max: 3.0
{"metric": "score", "value": 1.5}
step: (min: 172, avg: 248.9, max: 331
{"metric": "step", "value": 248.9}
{"metric": "run", "value":

loss: (min: 0.0005008923471905291, avg: 0.008228608981880825, max: 0.044516172260046005
{"metric": "loss", "value": 0.008228608981880825}
accuracy: (min: 0, avg: 0, max: 0
{"metric": "accuracy", "value": 0}
q: (min: -0.2650190971023403, avg: -0.0905046692377588, max: 0.06888752250000835
{"metric": "q", "value": -0.0905046692377588}
score: (min: 0.0, avg: 2.0, max: 6.0
{"metric": "score", "value": 2.0}
step: (min: 169, avg: 270.6, max: 479
{"metric": "step", "value": 270.6}
{"metric": "run", "value": 430}
----------------------------------------------------------------------------------------------------
RUN =  430
score: (min: 0.0, avg: 1.1, max: 3.0
{"metric": "score", "value": 1.1}
step: (min: 165, avg: 230.2, max: 343
{"metric": "step", "value": 230.2}
{"metric": "run", "value": 440}
----------------------------------------------------------------------------------------------------
RUN =  440
loss: (min: 0.0004908737610094249, avg: 0.008199497510504443, max: 0.1156257763504982
{"me

{"metric": "run", "value": 590}
----------------------------------------------------------------------------------------------------
RUN =  590
loss: (min: 0.00016576764755882323, avg: 0.00742964183317963, max: 0.11603011190891266
{"metric": "loss", "value": 0.00742964183317963}
accuracy: (min: 0, avg: 0, max: 0
{"metric": "accuracy", "value": 0}
q: (min: -0.1543915517104324, avg: -0.04405068916176198, max: 0.07936755130183883
{"metric": "q", "value": -0.04405068916176198}
score: (min: 0.0, avg: 1.4, max: 5.0
{"metric": "score", "value": 1.4}
step: (min: 160, avg: 247.6, max: 467
{"metric": "step", "value": 247.6}
{"metric": "run", "value": 600}
----------------------------------------------------------------------------------------------------
RUN =  600
score: (min: 0.0, avg: 1.0, max: 3.0
{"metric": "score", "value": 1.0}
step: (min: 158, avg: 223.5, max: 343
{"metric": "step", "value": 223.5}
{"metric": "run", "value": 610}
----------------------------------------------------------

loss: (min: 0.00012615110608749092, avg: 0.004130822925537359, max: 0.04071273282170296
{"metric": "loss", "value": 0.004130822925537359}
accuracy: (min: 0, avg: 0, max: 0
{"metric": "accuracy", "value": 0}
q: (min: -0.06752449668827466, avg: 0.00881082796275965, max: 0.09984852460504044
{"metric": "q", "value": 0.00881082796275965}
score: (min: 1.0, avg: 1.9, max: 3.0
{"metric": "score", "value": 1.9}
step: (min: 211, avg: 279.2, max: 358
{"metric": "step", "value": 279.2}
{"metric": "run", "value": 760}
----------------------------------------------------------------------------------------------------
RUN =  760


In [12]:
str(datetime.datetime.now().strftime('%Y-%m-%d'))

'2019-05-05'

In [15]:
TRAINING_FREQUENCY

4

# To be ignored

In [ ]:
class DDQNTrainer:
    def __init__(self, game_name, input_shape, action_space):
        
        self.input_shape = input_shape
        self.game_name = game_name
        self.action_space = action_space
        
        self.game_mode = "DDQN training"
        self.model_path = "./output/neural_nets/" + game_name + "/ddqn/" + self._get_date() + "/model.h5"
        self.logger_path = "./output/logs/" + game_name + "/ddqn/training/" + self._get_date() + "/"
        self.logger = Logger(self.game_name + " " + self.game_mode, self.logger_path)
        
        if os.path.exists(os.path.dirname(self.model_path)):
            shutil.rmtree(os.path.dirname(self.model_path), ignore_errors=True)
        os.makedirs(os.path.dirname(self.model_path))

        self.ddqn = ConvolutionalNeuralNetwork(self.input_shape, action_space).to(device)
        
        self.criteria = nn.MSELoss()
        self.optimizer = RMSprop(self.ddqn.parameters(), lr=0.00025, alpha = 0.95, eps=0.01)
        
        if os.path.isfile(self.model_path):
            self.ddqn.load_state_dict(torch.load(self.model_path))
        
        self.ddqn_target = ConvolutionalNeuralNetwork(self.input_shape, action_space).to(device)
#         .model
        self._reset_target_network()
        self.epsilon = EXPLORATION_MAX
        self.memory = []
    
    def _save_model(self):
        torch.save(self.ddqn.state_dict(),self.model_path)
#         self.ddqn.save_weights(self.model_path)
        
    def _get_date(self):
        return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
        
    def save_run(self, score, step, run):
        self.logger.add_score(score)
        self.logger.add_step(step)
        self.logger.add_run(run)

    def move(self, state):
        if np.random.rand() < self.epsilon or len(self.memory) < REPLAY_START_SIZE:
            return random.randrange(self.action_space)
        
        q_values = self.ddqn(np.expand_dims(np.asarray(state).astype(np.float64), axis=0), batch_size=1)
        return np.argmax(q_values[0].detach().cpu().numpy())

    def remember(self, current_state, action, reward, next_state, terminal):
        self.memory.append({"current_state": current_state,
                            "action": action,
                            "reward": reward,
                            "next_state": next_state,
                            "terminal": terminal})
        if len(self.memory) > MEMORY_SIZE:
            self.memory.pop(0)

    def step_update(self, total_step):
        if len(self.memory) < REPLAY_START_SIZE:
            return

        if total_step % TRAINING_FREQUENCY == 0:
            loss, accuracy, average_max_q = self._train()
            self.logger.add_loss(loss.item())
            self.logger.add_accuracy(accuracy)
            self.logger.add_q(average_max_q)

        self._update_epsilon()

        if total_step % MODEL_PERSISTENCE_UPDATE_FREQUENCY == 0:
            self._save_model()

        if total_step % TARGET_NETWORK_UPDATE_FREQUENCY == 0:
            self._reset_target_network()
            print('{{"metric": "epsilon", "value": {}}}'.format(self.epsilon))
            print('{{"metric": "total_step", "value": {}}}'.format(total_step))

    def _train(self):
        
        self.optimizer.zero_grad()
        batch = np.asarray(random.sample(self.memory, BATCH_SIZE))
        if len(batch) < BATCH_SIZE:
            return

        current_states = []
        actions = []
        rewards = []
        next_states = []
        terminals = []
        
        q_values = []
        max_q_values = []
        
#         print(batch)
        for entry in batch:
            current_state = np.expand_dims(np.asarray(entry["current_state"]).astype(np.float64), axis=0)
            current_states.append(current_state)
            next_state = np.expand_dims(np.asarray(entry["next_state"]).astype(np.float64), axis=0)
            
            next_states.append(next_state)
            rewards.append(entry['reward'])
            actions.append(entry['action'])
            terminals.append(entry['terminal'])
            
        non_final_mask = torch.tensor(terminals)==False
        non_final_next_states = np.asarray(next_states).squeeze()
        
        state_batch = np.asarray(current_states).squeeze()
        action_batch = np.asanyarray(actions).squeeze()
        reward_batch = np.asarray(rewards).squeeze()
#         print(torch.from_numpy(action_batch).unsqueeze(dim=1).size(),torch.Size([32, 1]))
        state_action_values = self.ddqn(state_batch).gather(1, torch.from_numpy(action_batch).unsqueeze(dim=1).to(device))
    
#         next_state_values = torch.zeros(BATCH_SIZE, device=device)
        next_state_values = np.zeros(BATCH_SIZE)
        next_state_values[non_final_mask] = np.max(self.ddqn_target(non_final_next_states).detach().cpu().numpy())
#         .max(1)[0].detach().cpu()    
    
        expected_state_action_values = (next_state_values * GAMMA) + reward_batch
        
        loss = self.criteria(state_action_values, expected_state_action_values.unsqueeze(1))
        loss.backward()
        self.optimizer.step()
        
#         print(state_action_values.size(),[32, 4])
#         print(state_action_values.size(),torch.Size([32, 1]))
#         print(non_final_next_states.squeeze().size(), torch.Size([32, 4, 84, 84]))
        
    
    
    
        
#             next_state_prediction = self.ddqn_target(next_state).detach().cpu().numpy().ravel()
# #             print(next_state_prediction.shape, (4,))
            
#             next_q_value = np.max(next_state_prediction)
#             q = list(self.ddqn(current_state)[0].detach().cpu().numpy())
# #             print(len(q), 4)
#             if entry["terminal"]:
#                 q[entry["action"]] = entry["reward"]
#             else:
#                 q[entry["action"]] = entry["reward"] + GAMMA * next_q_value
#             q_values.append(q)
#             max_q_values.append(np.max(q))
        
#         model_out = self.ddqn(np.asarray(current_states).squeeze())
#         loss = self.criteria(model_out, torch.from_numpy(np.array(q_values)).float().to(device))
#         loss.backward()
#         self.optimizer.step()
    

        accuracy = 0
        return loss, accuracy, mean(max_q_values)

    def _update_epsilon(self):
        self.epsilon -= EXPLORATION_DECAY
        self.epsilon = max(EXPLORATION_MIN, self.epsilon)

    def _reset_target_network(self):
        self.ddqn_target.load_state_dict(self.ddqn.state_dict())